In [1]:
from sage.combinat.vector_partition import IntegerVectorsIterator
from collections import Counter
from IPython.display import Markdown

from functools import reduce
import itertools


def clr_print(t,color='red'):
    display(Markdown('<span style="color:'+str(color)+'">' + t + '</span>'))

def size_print(t, size = '4em'):
    display(Markdown('<span style="font-size:'+str(size)+'">' + t + '</span>'))

class QuiverRoots(SageObject):
    """
    Keeps track of roots, decompositions for 
    quiver varieties with stability parameters
    based on the Crawley-Boevey description
    """
    def __init__(self, C):
        """
        Initialize for a quiver with Cartan matrix C
        This is required to include the framing vertex infty 
        after reduction to the W= 0 case
        """
        self.C = C
        self.rk = C.dimensions()[0]
        self.v = VectorSpace(QQ,self.rk, inner_product_matrix=C )
        self.pair = lambda a,b : vector(a)*C*vector(b)
        self.p = lambda a : 1 - 1/2*self.pair(a,a)
        
    def unframed_stab(self,v, θ):
        """
        given a len n-1 stability vector θ for framed quiver reps,
        spits out (-θ(v), θ),
        which is the appropriate stability condition for the CB reduction to 
        unframed case
        """
        return(vector([-vector(v[1:])*vector(θ)] + list(θ)))
        
    def has_connected_support(self, v):
        """
        checks if dimension vector v has connected support on the
        dyinkin diagram associated to C
        """
        listv = list(v)
        support = [i for i in range(self.rk) if listv[i] != 0]
        a_component = set([support[0]])
        checked = a_component
        unchecked = set(support[1:])
        adjacents_in_support = lambda i : [a for a in range(self.rk) if 
                                           a != i and 
                                           C[i,a] != 0 and 
                                           a in support]
        current = support[0]
        while len(checked) < len(support):
            checked.add(current)
            unchecked.discard(current)
            a_component.update(adjacents_in_support(current))
            unchecked_in_cpt = unchecked.intersection(a_component)
            if len(unchecked_in_cpt) == 0:
                break
            current = unchecked_in_cpt.pop()
        if len(a_component) == len(support):
            return(True)
        else:
            return(False)
        
        
    def R_v(self, v, connected_support = True):
        """
        returns positive roots less than v, without the connected 
        support condition if  connected_support is set to False
        """
        if connected_support == True:
            return(α for α in IntegerVectorsIterator(list(v)) if self.pair(vector(α), vector(α)) <= 2 and vector(α) != 0 and 
                  self.has_connected_support(α))
        else: 
            return(α for α in IntegerVectorsIterator(list(v)) if self.pair(α, α) <= 2)
    
    def R_θ_v(self,θ,v, connected_support = True):
        """
        Returns positive roots α less than v, with θ(α) = 0 
        again without connected support condition if  connected_support is set to False
        """
        rv = self.R_v(v)
        return([α for α in rv if vector(θ)*vector(α) == 0])
    
    def candidate_decompositions(self, θ, α):
        """
        Returns vector partitions of α
        such that each part is in R_θ
        """
        VPs = VectorPartitions(α)
        def each_part_in_R_θ(l):
            for p in l:
                if vector(θ)*vector(p) != 0:
                    return(False)
            return(True)
        return(vp for vp in VPs if each_part_in_R_θ(vp))
    
    def Σ_θ_v(self, θ, v):
        """
        Returns real possible decomposition factor options
        of roots less than v
        the connected support condition is used
        """
        p = self.p
        def in_Σ(α):
            cd = list(self.candidate_decompositions(θ, α))
            for bs in cd:
                if len(bs) > 1 and (sum(p(β) for β in bs) >= p(α)):
                    return(False)
            return(True)
        return(α for α in self.R_θ_v(θ, v) if in_Σ(α))
    
    def uncollected_decompositions(self, θ, v):
        """
        Returns vector partitions of v all of whose 
        parts are in Σ_θ_v
        A decomposition has some of the same parts collected
        the connected support condition is used
        """
        candidates = self.candidate_decompositions(θ, v)
        Σ = list(self.Σ_θ_v(θ, v))
        #print(f"Σ = {Σ}")
        def each_part_in_Σ(l):
            for p in l:
                if not (p in Σ):
                    return(False)
            return(True)
        return(d for d in candidates if each_part_in_Σ(d))
    
    def completely_collected_decompositions(self,θ,v):
        """
        returns lists of tuples
        [(β_1,n_1), ..., (β_k, n_k)]
        giving decompositions where 
        i \neq j implies β_i \neq β_j
        """
        out = []
        for d in self.uncollected_decompositions(θ,v):
            td = [tuple(a) for a in d]
            out.append(list(Counter(td).items()))
        return(out)
            
        return(list(Counter(self.uncollected_decompositions(θ, v)).items()))
    
    def decompositions(self, θ, v):
        r"""
        returns all decompositons 
        [(β_1,n_1), ..., (β_k, n_k)]
        corresponding to 
        $\oplus_i \beta_i^{\oplus n_i}$
        as dimension vectors.
        """
        completely_collected = self.completely_collected_decompositions(θ, v)
        out = []
        for decomp in completely_collected:
            imaginary_parts = [(β,n) for (β,n) in decomp if self.p(β) > 0]
            unrefined_reals = [(β, n) for (β, n) in decomp if self.p(β) <= 0]
            new = unrefined_reals
            refinements_of_βn = lambda β, n: [[(β, λi) for λi in λ] for λ in Partitions(n)] # lists refinements of this summand
            endings = [list(itertools.chain(*a)) for a in itertools.product(*[refinements_of_βn(β, n) for (β, n) in imaginary_parts])]
            out += [new + e for e in endings]
        return(out)
        
    
    def stratum_dimension(self, decomp):
        """
        takes a decomposition
        [β_1^n_1, ..., β_k^n_k]
        given as a list of tupes
        [(β_1,n_1), ..., (β_k, n_k)]
        and returns
        2*sum (p(β_i)) 
        the dimension of the stratum. 
        If β_i is REAL it can only show up once for this formula
        to make sense
        """
        βs = [β for (β, n) in decomp]
        for β in [β for β in βs if βs.count(β) > 1]:
            assert self.p(β) > 0, "real roots β in decomposition must be unique"
        return(2*sum(self.p(β) for β in βs))
    
    def stratum_ext_quiver(self, decomp, θ, θs, framed = True):
        r"""
        gives the information about the local 
        structure of the singularity at the condition 
        θ

        INPUT : 

        - ``decomp`` -- [(β_1,n_1), ..., (β_k, n_k)]; the decomposition
          describing the stratum

        - ``θ`` -- vector; gives stability condition on the wall
        
        - ``θs`` -- list of vectors; which stability conditions adjacent to 
          this wall we are considering
          
        - ``framed`` -- bool (default: True); Whether to use the framed equivalent
          quiver or the Crawley-Boevey one

        OUTPUT: (Q,v,l, [ρs]) where Q is the graph of th ext quiver
         v is the dimension vector
         l is an integer 
         and ρ_i in the list [ρs] is 
         the analogue of θi so the map M_θ_i -> M_θ is analytically locall
         C^l x M_ρ(Q,v) -> M_0(Q,v) at a point of M_θ in the stratum with decomposition
         decomp

        EXAMPLES: 

        This exmaple shows an example of the above, finding all
        strata for affine A_1 quiver and a v = (1,2) w = (1,0) ::

            sage: C = Matrix([[2,-1,0],[-1,2,-2],[0,-2,2]])
            sage: QR = QuiverRoots(C)
            sage: v = (1,1,2)
            sage: θ = QR.unframed_stab(v, (1,0))
            sage: for dc in QR.decompositions(θ, v):
            sage:     print(f"stratum {dc} has dim {QR.stratum_dimension(dc)}")
            sage:     (Q,v,l,q) = QR.stratum_ext_quiver(dc, θ, [])
            sage:     print(f"v = {v}, l = {l}")
            sage:     show(Q, figsize=1)
            stratum [((0, 0, 1), 2), ((1, 1, 0), 1)] has dim 0
            v = [1, 2], l = 0
            <Graphics object> 
            stratum [((0, 0, 1), 1), ((1, 1, 1), 1)] has dim 2
            v = [1, 1], l = 2
            <Graphics object> 

        NOTE ::

            this code doesn't spit out [ρi] yet 
            We spit out the dimension v of the CB "unframed" quvier 
            even if framed is True
        """
        dc = sorted(decomp, reverse=True) #makes sure vertex 1 is framing vertex b/c first element will be
        #(1,...),  others (0,...)
        k = len(dc)
        if framed == False:
            self_loops = lambda i : [lbl(i)]*int(self.p(dc[i][0])) if i != 0 else []
            edges_at = lambda i :  self_loops(i) + list(itertools.chain(*([lbl(j)]*(-self.pair(dc[i][0], dc[j][0]) ) for j in range(k) if j != i)))
            lbl = lambda t : '∞' if t == 0 else t
            Q = Graph({lbl(i):edges_at(i) for i in range(k)})
            l = 2*self.p(dc[0][0])
            v = [n for (β, n) in dc]
            ρs = []
            return((Q,v,l,ρs))
        else:
            v = [n for (β, n) in dc]
            lbl = lambda i : "v_"+ str(i) + "=" + str(v[i])
            unframing_edges = lambda i : [lbl(i)]*int(self.p(dc[i][0])) + list(itertools.chain(*([lbl(j)]*(-self.pair(dc[i][0], dc[j][0]) ) for j in range(1,k) if j != i)))
            framing_g_dict = {str(-self.pair(dc[0][0], dc[i][0])) + "dim framing" : [lbl(i)] for i in range(1,k)}
            framing_g_dict.update({lbl(i):unframing_edges(i) for i in range(1,k)})
            Q = Graph(framing_g_dict)
            l = 2*self.p(dc[0][0])
            ρs = []
            return((Q,v,l,ρs))
    
    def generic_in_R_perp(self, v, R):
        pass

<span style="color:red">uh oh</span>

In [1]:
def plot_quiver_stability_space(QR,v,lvl=1):
    """
    This is actually unique to affine A_2...
    """
    assert QR.rk == 4; "must be framed 2d affine quiver, e.g. framed affine A_2"
    bound = max(v) + 2
    R = [a for a in QR.R_v(v) if a[0] == 0]
    Rp = [a[1:] for a in R if gcd(a) == 1] #removed reducancy
    Rpp = [a for a in Rp if QR.has_connected_support(tuple(vector(v) - vector([0] + a)))]
    Rppp = [(t[0], [t[1] - t[0], t[2] - t[0]]) for t in Rpp] # break into kδ + finite root
    for (k,a) in Rppp:
        if list(a) in [[0,1],[0,-1],[1,0],[-1,0]]:
            root_to_max_stratum_dimension(QR,v,k,a)
    # e_1 = (1,0), e2 = (-1/2, sqrt(3)/2)
    finite_root_coords = lambda a : (a[0] - 0.5*a[1], N(sqrt(3))*0.5*a[1])
    var("θ1, θ2")
    print(Rppp)
    eq = lambda a,k : a[0]*θ1 + a[1]*θ2 + k
    k_zero_red = sum(implicit_plot(eq(finite_root_coords(a), k) , (θ1, -bound,bound), (θ2, -bound,bound), linewidth = 0.3, color = 'red' ) for (k,a) in Rppp if k == 0)
    k_non_zero_blue = sum(implicit_plot(eq(finite_root_coords(a), k) , (θ1, -bound,bound), (θ2, -bound,bound), linewidth = 0.2, color = 'blue' ) for (k,a) in Rppp if k != 0)
    return(k_zero_red + k_non_zero_blue)

def root_to_max_stratum_dimension(QR,v, k, a):
    assert list(a) in [[0,1],[0,-1],[1,0],[-1,0]]; 'only want these ones'
    simple_dim = (k, k+a[0], k+a[1])
    if a[1] == 0:
        θ = (1 + a[0]*k - 3*sum(v), -a[0]*k, 3*sum(v))
    else:
        θ = (1 + a[1]*k - 3*sum(v), 3*sum(v),-a[1]*k)
    unframedθ = QR.unframed_stab(v, θ)
    strata = QR.decompositions(unframedθ, v)
    dms = [(st, QR.stratum_dimension(st)) for st in strata]
    print(f"v = {v},simple = {simple_dim}, θ = {θ}, stratadims = \n{dms}")
    dmv = QR.stratum_dimension([(v, 1)])
    for (st,dim) in dms:
        if dim == dmv -2:
            clr_print(f"stratum{st} has divisorial dim {dim}", 'red')
        